In [55]:
import cv2
import numpy as np
import os

def detect_green_areas(input_dir, output_dir, min_area_fraction=0.001):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    counter = 1
    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):

        if filename.endswith(('.jpg', '.jpeg', '.png')):  # You can add more file extensions if needed
            image_path = os.path.join(input_dir, filename)
            
            # Read the image
            image = cv2.imread(image_path)

            # Convert image to HSV
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Define the lower and upper bounds for green color
            lower_green = np.array([5, 45, 45])
            upper_green = np.array([80, 255, 255])

            # Create a binary mask where green regions are white
            mask = cv2.inRange(hsv, lower_green, upper_green)

            # Morphological operations to remove noise
            kernel = np.ones((3, 3), np.uint8)
            mask = cv2.erode(mask, kernel, iterations=1)
            mask = cv2.dilate(mask, kernel, iterations=2)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            

            # Calculate minimum area based on image dimensions
            min_area = min_area_fraction * image.shape[0] * image.shape[1]

            # Find connected components and their statistics
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)

            # Filter out components based on their area
            for i in range(1, num_labels):  # Exclude background label (0)
                if stats[i, cv2.CC_STAT_AREA] < min_area:
                    mask[labels == i] = 0  # Set pixels of small components to 0

            # Create a white background image
            white_background = np.ones_like(image) * 255

            # Merge the original image and the white background using the mask
            result = cv2.bitwise_and(image, image, mask=mask)
            result += cv2.bitwise_and(white_background, white_background, mask=cv2.bitwise_not(mask))

            # Save the result to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, result)
            print(counter)
            counter += 1

# Paths to the directories
input_directory = r"D:\Projects\Weed Detection\data\Euphorbia helioscopia\OPPD-master-DATA-images_plants-EPHHE\DATA\images_plants\EPHHE"
output_directory = r"D:\Projects\Weed Detection\data\Euphorbia helioscopia\processed"

# Call the function with min_area_fraction set to 0.001 (adjust as needed)
detect_green_areas(input_directory, output_directory, min_area_fraction=0.009)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
